In [1]:
import time
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras
from keras import models
from keras import layers

%matplotlib inline

Using TensorFlow backend.
/home/rsouza/Documents/envs/python_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rsouza/Documents/envs/python_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rsouza/Documents/envs/python_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rs

In [3]:
#trainpath = '../data/kaggle-train-small/audio/'
trainpath = '../data/kaggle-train/audio/'

In [ ]:
start = time.time()

files = os.listdir(trainpath)
#files = files[0:10000]
stft = []
rmse = []
spec_cent = []
spec_bw = []
rolloff = []
zcr = []
mfcc = []

for idx, f in enumerate(files):
    filename = os.path.join(trainpath, str(f))
    y, sr = librosa.load(filename, mono=True, duration=120)
    stft.append(librosa.feature.chroma_stft(y=y, sr=sr))
    #rmse.append(librosa.feature.rms(y=y))
    #spec_cent.append(librosa.feature.spectral_centroid(y=y, sr=sr))
    #spec_bw.append(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    #rolloff.append(librosa.feature.spectral_rolloff(y=y, sr=sr))
    #zcr.append(librosa.feature.zero_crossing_rate(y))
    #mfcc.append(librosa.feature.mfcc(y=y, sr=sr))
    if idx%5000 == 0:
        print(f'processing file: {idx}')
    
stft = np.array(stft)
#rmse = np.array(rmse)
#spec_cent = np.array(spec_cent)
#rolloff = np.array(rolloff)
#zcr = np.array(zcr)
#mfcc = np.array(mfcc)

np.save('./stft.numpy', stft)
#np.save('./rmse.numpy', rmse)
#np.save('./spec_cent.numpy', spec_cent)
#np.save('./rolloff.numpy', rolloff)
#np.save('./zcr.numpy', zcr)
#np.save('./mfcc.numpy', mfcc)

    
print("Extracting the features took " + str((time.time()-start)/60) + "mins")

processing file: 0:


/home/rsouza/Documents/envs/python_env/lib/python3.6/site-packages/librosa/core/pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


processing file: 1000:
processing file: 2000:
processing file: 3000:
processing file: 4000:
processing file: 5000:
processing file: 6000:
processing file: 7000:
processing file: 8000:
processing file: 9000:
processing file: 10000:
processing file: 11000:
processing file: 12000:
processing file: 13000:
processing file: 14000:
processing file: 15000:
processing file: 16000:
processing file: 17000:
processing file: 18000:
processing file: 19000:
processing file: 20000:
processing file: 21000:
processing file: 22000:
processing file: 23000:
processing file: 24000:
processing file: 25000:
processing file: 26000:
processing file: 27000:
processing file: 28000:
processing file: 29000:
processing file: 30000:
processing file: 31000:
processing file: 32000:
processing file: 33000:
processing file: 34000:
processing file: 35000:
processing file: 36000:
processing file: 37000:
processing file: 38000:
processing file: 39000:
processing file: 40000:
processing file: 41000:
processing file: 42000:
p

In [ ]:
stft = np.load('./stft.numpy')
rmse = np.load('./rmse.numpy')
spec_cent = np.load('./spec_cent.numpy')
rolloff = np.load('./rolloff.numpy')
zcr = np.load('./zcr.numpy')
mfcc = np.load('./mfcc.numpy')

In [ ]:
df_train = pd.DataFrame(files, columns=['filename'])
df_train['audiotype'] = df_train['filename'].str.extract(pat = '_([a-z]*)_')
df_train['instrument'] = df_train['filename'].str.extract(pat = '^([a-z]*)_')
df_train['chroma_stft_mean'] = np.mean(stft, axis=(1,2))
df_train['chroma_stft_std'] = np.std(stft, axis=(1,2))
df_train['rmse_mean'] = np.mean(rmse, axis=(1,2))
df_train['rmse_std'] = np.std(rmse, axis=(1,2))
df_train['spec_cent_mean'] = np.mean(spec_cent, axis=(1,2))
df_train['spec_cent_std'] = np.std(spec_cent, axis=(1,2))
df_train['spec_bw_mean'] = np.mean(spec_bw, axis=(1,2))
df_train['spec_bw_std'] = np.std(spec_bw, axis=(1,2))
df_train['rolloff_mean'] = np.mean(rolloff, axis=(1,2))
df_train['rolloff_std'] = np.std(rolloff, axis=(1,2))
df_train['zcr_mean'] = np.mean(zcr, axis=(1,2))
df_train['zcr_std'] = np.std(zcr, axis=(1,2))
for j in range(0, 20):
    df_train[f' mfcc_mean{j}'] = np.mean(mfcc[:,j,:], axis=1)
    df_train[f' mfcc_std{j}'] = np.std(mfcc[:,j,:], axis=1)

df_train.dropna(inplace=True)
df_train.to_pickle('./df_train.pkl')

In [ ]:
df_train = pd.read_pickle('./df_train.pkl')
df_train.head(20)

In [ ]:
data = df_train[[c for c in df_train.columns if c not in ['filename', 'audiotype']]]

In [ ]:
# normalizing
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:,2:], dtype = float))
print(X[0])

In [ ]:
# defining target
encoder = LabelEncoder()
y = encoder.fit_transform(data['instrument'])
print(y[0:5])

In [ ]:
# spliting of dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
# creating a model
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
              
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                    
# calculate accuracy
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

# predictions
predictions = model.predict(X_test)
np.argmax(predictions[0])

In [ ]:
testpath = '../data/kaggle-test/audio/'

In [ ]:
start = time.time()

files = os.listdir(testpath)
stft = []
rmse = []
spec_cent = []
spec_bw = []
rolloff = []
zcr = []
mfcc = []

for f in files:
    filename = os.path.join(testpath, str(f))
    y, sr = librosa.load(filename, mono=True, duration=120)
    stft.append(librosa.feature.chroma_stft(y=y, sr=sr))
    rmse.append(librosa.feature.rms(y=y))
    spec_cent.append(librosa.feature.spectral_centroid(y=y, sr=sr))
    spec_bw.append(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    rolloff.append(librosa.feature.spectral_rolloff(y=y, sr=sr))
    zcr.append(librosa.feature.zero_crossing_rate(y))
    mfcc.append(librosa.feature.mfcc(y=y, sr=sr))
    
stft = np.array(stft)
rmse = np.array(rmse)
spec_cent = np.array(spec_cent)
rolloff = np.array(rolloff)
zcr = np.array(zcr)
mfcc = np.array(mfcc)
    
print("Extracting the features took " + str((time.time()-start)/60) + "mins")

In [ ]:
df_predict = pd.DataFrame(files, columns=['id'])
df_predict['id'] = df_predict['id'].str.extract(pat = '([0-9]*).wav')
df_predict['id'] = pd.to_numeric(df_predict['id'])
#df_predict['instrument'] = df_predict['filename'].str.extract(pat = '^([a-z]*)_')
df_predict['chroma_stft_mean'] = np.mean(stft, axis=(1,2))
df_predict['chroma_stft_std'] = np.std(stft, axis=(1,2))
df_predict['rmse_mean'] = np.mean(rmse, axis=(1,2))
df_predict['rmse_std'] = np.std(rmse, axis=(1,2))
df_predict['spec_cent_mean'] = np.mean(spec_cent, axis=(1,2))
df_predict['spec_cent_std'] = np.std(spec_cent, axis=(1,2))
df_predict['spec_bw_mean'] = np.mean(spec_bw, axis=(1,2))
df_predict['spec_bw_std'] = np.std(spec_bw, axis=(1,2))
df_predict['rolloff_mean'] = np.mean(rolloff, axis=(1,2))
df_predict['rolloff_std'] = np.std(rolloff, axis=(1,2))
df_predict['zcr_mean'] = np.mean(zcr, axis=(1,2))
df_predict['zcr_std'] = np.std(zcr, axis=(1,2))
for j in range(0, 20):
    df_predict[f' mfcc_mean{j}'] = np.mean(mfcc[:,j,:], axis=1)
    df_predict[f' mfcc_std{j}'] = np.std(mfcc[:,j,:], axis=1)
    
df_predict.sort_values(by='id', inplace=True)
df_predict.head()

In [ ]:
df_predict.info()

In [ ]:
df_predict.dropna(inplace=True)
testdata = df_predict[[c for c in df_predict.columns if c not in ['id']]]
X = scaler.fit_transform(np.array(testdata.iloc[:,1:], dtype = float))
X.shape

In [ ]:
familyPredictions = model.predict(X)

In [ ]:
with open('SVM-time-submission.csv', 'w', newline='') as writeFile:
    fieldnames = ['Id', 'Predicted']
    writer = csv.DictWriter(writeFile, fieldnames=fieldnames, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writeheader()
    for index in range(len(testDataset)):
        writer.writerow({'Id': index, 'Predicted': familyPredictionStrings[index]})